In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
train_df = pd.read_csv('../input/train.tsv', sep='\t')

In [ ]:
train_df.head()

In [ ]:
from keras.preprocessing import text

In [ ]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = text.Tokenizer(filters='')
tokenizer.fit_on_texts(train_df.iloc[:,2])

seq = tokenizer.texts_to_sequences(train_df.iloc[:,2])

word_ind = tokenizer.word_index
X = pad_sequences(seq, maxlen=50, padding='post')

In [ ]:
from sklearn.cross_validation import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, train_df.iloc[:,3], train_size=0.8)

In [ ]:
from keras.layers import Embedding, Convolution1D, MaxPooling1D, Flatten, LeakyReLU, Dense, BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, rmsprop, adam
from keras.callbacks import EarlyStopping

In [ ]:
from keras.layers import CuDNNLSTM

In [ ]:
model = Sequential()
model.add(Embedding(len(word_ind) + 1, 300,
                    input_length=50))
model.add(CuDNNLSTM(300, return_sequences=True))
model.add(CuDNNLSTM(300, return_sequences=True))
model.add(Flatten())
model.add(Dense(1))

In [ ]:
#65.22% !!
sgd = SGD(nesterov=True)
rms = rmsprop()
model.compile(rms, loss='mse', metrics=['accuracy'])
es = EarlyStopping(patience=2, monitor='val_acc')

In [ ]:
model.fit(X_train, np.array(y_train), batch_size=32, epochs=50,
          validation_split=0.25, callbacks=[es])

In [ ]:
model.evaluate(X_val, y_val)

In [ ]:
test_data = pd.read_csv('../input/test.tsv', sep='\t')
sample = pd.read_csv('../input/sampleSubmission.csv')

In [ ]:
test_seq = tokenizer.texts_to_sequences(test_data.iloc[:,2])
test_seq = pad_sequences(test_seq, maxlen=50, padding='post')

pred = np.round(model.predict(test_seq))

In [ ]:
def ranged_five(x):
    if x > 4:
        return 4
    elif x < 0:
        return 0
    else:
        return x

In [ ]:
vranged_five = np.vectorize(ranged_five)
pred = (vranged_five(pred))
pred = np.reshape(pred, pred.shape[0])

In [ ]:
sample.iloc[:,1] = pred.astype(np.int64)

In [ ]:
sample.to_csv('submission2.csv', index=False)